This notebook is adapted from [a lesson from the 2017 KIPAC/StatisticalMethods course](https://github.com/KIPAC/StatisticalMethods/blob/2017winter/chunks/generative_models.ipynb), (c) 2017 Adam Mantz and Phil Marshall. The GPLv2 license applies.

# Generative Models

Goals:
* Introduce generative models in the context of mocking data and inference
* Introduce probabilistic graphical models as a tool for model visualization
* Practice building some simple models

## Further reading

* Ivezic et al, Sections 3.3 and 3.7
* [Bishop, "Pattern Recognition and Machine Learning,"](https://www.amazon.com/Pattern-Recognition-Learning-Information-Statistics/dp/0387310738) Sections 8.1 and 8.2

A **generative model** formalizes our understanding of how a data set comes to exist, including
* physical processes happening out there in the Universe
* instrumental effects and the measurement process
* any computations done prior to calling the result a "data set"

In other words, it's what we need in order to generate a mock data set.

To actually generate mock data, we need to specify the **sampling distribution**, $P(\mathrm{data}|\mathrm{model})$. This PDF is the mathemetical expression of our generative model.
* It shows up directly in Bayes Theorem, and ideally (as a function of model parameters) as the likelihood function in maximum-likelihood fitting.

What are generative models useful for?

* Performing inference: constructing the *sampling distribution* or *likelihood function*

* Testing inference: does our analysis, run on mock data, recover the input model?

* Checking inferences: do mock data generated from a fitted model resemble the real data?

## The Sampling Distribution

* The noisy, mock data are drawn from a PDF known as the "sampling distribution"

* In the case of an X-ray image pixel, containing mock counts $N_k$, this PDF is $P(N_k|\mu_k,H)$

* $\mu_k$ is a parameter, the expected number of counts in the $k^{th}$ pixel

* $H$ is the list of assumptions that defines our model

> The sampling distribution is sometimes called the error distribution, because it captures the statistical properties of the "random errors" in the data. Drawing from the sampling distribution "adds noise" to the simulation.

## Conditional probability

* $P(N_k|\mu_k,H)$ is pronounced "the probabilty for $N_k$ given $\mu_k$ and $H$"

* This means that if you know what the value of $\mu_k$ is, you can _draw_ a sample value of $N_k$ from $P$ - since  $H$ tells you what the functional form of $P$ is.

## Sampling in practice

* In general, sampling from a PDF is difficult

* For certain standard distributions, however, there are fast algorithms

In [ ]:
import scipy.stats

P = scipy.stats.poisson(mu=3)
P.rvs(size=10)

## Sampling in practice

* [numpy.random]() and [scipy.stats]() are two useful libraries for drawing samples from PDFs

* You may have encountered some of these routines as "random number generators"

* Sampling from a PDF means generating random numbers from that distribution

## Choosing the input model parameters

* When testing, we often want to assert a set of input model parameters $\theta$, and then see what they produce

* Testing at large scale might involve generating many datasets, with different inputs. In this case we might want to sample from a *plausible distribution of input parameters*

* In practice: choose a particular standard probability distribution for $\theta$ and sample from it

## Deterministic relationships

* Often, our model provides a deterministic relationship between parameters: if you know $\theta$, then you know $\mu$

* In our X-ray image case, $\theta$ could be the set of parameters that describe the gas temperature and density profiles of a spherically symmetric cluster of galaxies with known centroid position

* $\mu(\theta)$ would then be some complicated function that took the cluster parameters $\theta$ as input, and predicted the expectation value of the counts at any pixel position.

A **probabilistic graphical model** (PGM) is a very useful way of visualizing a generative model.
* They sketch out the procedure for how one would generate mock data in practice.
* They illustrate the interdependence of model parameters, and the dependence of data on parameters.
* They also (therefore) represent a conditional factorization of the sampling distribution.

**Many, many** mistakes can be avoided by sketching out a PGM at the outset of a statistical analysis.

Technically, a PGM is a type of *directed acyclic graph*, where **nodes** and **edges** represent parts of the model.

Let's look at a very simple example...

<table><tr width=90%>
<td><img src="../graphics/tour_cluster_image.png" height=300></td>
<td><img src="../graphics/tour_cluster_image_zoom.png" height=300></td>
</tr></table>

Our measurement is the number of counts in each pixel. Here is a generative model:

* There's an object emitting light, whose properties are parametrized by $\theta$.

* From $\theta$, we can determine the average flux falling on a given pixel $k$, $F_k$.

* Given the exposure time of our observation, $T$, and some conversion factors, $F_k$ determines the average number of counts expected, $\mu_k$.

* The number of counts measured, $N_k$, is a Poisson draw, given the average $\mu_k$.

Notice that the model was described in terms of conditional relationships.
* $\theta \Rightarrow F_k$
* $F_k,T \Rightarrow \mu_k$
* $N_k \sim \mathrm{Poisson}(\mu_k)$

The PGM will do the same, visually.

<img src="../graphics/pgms_pixelcounts.png">

Ingredients of a PGM:
* **Nodes** represent parameters
* **Edges** represent conditional relationships
* **Plates** represent repeated model components whose contents are conditionally independent

Types of nodes:
* **Circles** represent a PDF. This parameter is a *stochastic* function of the parameters feeding into it.
* **Points** represent a delta-function PDF. This parameter is a *deterministic* function of the parameters feeding into it.
* **Double circles** (or shading) indicate measured data. They are stochastic in the context of generating mock data, but fixed in the context of parameter inference.

Q: What is this PGM telling us?

<img src="../graphics/pgms_pixelcounts.png">

Q: How are these PGMs different, and what does the difference mean?

<table><tr><td>
<img src="../graphics/pgms_pixelcounts.png">
</td><td>
<img src="../graphics/pgms_pixelcounts2.png">
</td></tr></table>

## Probabilistic Graphical Models

* This procedure for simulating a mock X-ray image dataset can be usefully illustrated with a _directed acyclic graph_ called a "Probabilistic Graphical Model"

* In the present context these provide something like a "flow diagram" showing how we might draw a sample mock dataset from our model

* Let's look at the PGM for a simple X-ray image simulation 


## One pixel, fixed inputs

<img src="../graphics/pgms_one_pixel_input_fixed.png">

## PGM pieces

* Each _node_ in the graph represents a PDF, for the variable labeled inside it

* Each _edge_ (arrow) in the graph represents a conditional dependence

* Deterministic relationships are indicated by "fixed" variables represented by solid points

> The fixed nodes are _also_ PDFs: fixing a parameter is the same as sampling it from a delta function PDF

## One pixel, sampled inputs

<img src="../graphics/pgms_one_pixel_input_sampled.png" width=70%>

## PGM interpretation

* Following the arrows, the network of conditional dependences shows you how to go about simulating mock data ($N_k$)

> For example: 
1. Draw a sample $\theta$ vector;
2. Compute $\mu_k$ from it;
3. Draw an $N_k$ given that $\mu_k$

By mapping the conditional dependences of a model, PGMs illustrate how to factorize the sampling distribution:

$P(\theta) \prod_k P(N_k|\mu_k)P(\mu_k|F_k,T)P(F_k|\theta)$

<img src="../graphics/pgms_pixelcounts.png">

In this case, some PDFs are delta functions, so we can simplify:

$P(\theta) \prod_k P(N_k|\mu_k)P(\mu_k|F_k,T)P(F_k|\theta)$

$= \underbrace{P(\theta)} ~ \underbrace{\prod_k P\left[N_k|\mu_k(\theta,T)\right]}$

$= \mathrm{prior}(\theta) ~\times~ (\mathrm{sampling~distribution~of~}\vec{N})$

## PGM interpretation

* The graph is also an illustration of a particular factorisation of a joint probability distribution: the PDF for every variable in the model 

* For example: 

$\;\;\;\;\;\;\;P(N_k|\mu_k,H)\;P(\mu_k|\theta_k,H)\;P(\theta_k|H) =  P(N_k,\mu_k,\theta_k|H)$

> Note that the dependence of $N_k$ on $\theta$ has been dropped here: $N_k$ is only dependent on $\theta$ through $\mu_k$, or if you prefer, $N_k$ is _conditionally independent_ of $\theta$ given $\mu$

**Exercise**

<table><tr>
<td><img src="../graphics/pgms_a-c-d.png"></td>
<td><img src="../graphics/pgms_c-y-d.png"></td>
</tr></table>

* On your own, write down the probability expressions illustrated by these two graphs. 
* Then, discuss their meaning with your neighbor, and prepare to report back to the class.

## Simulating the whole image

* Our model for the cluster allows us to predict all of the image pixel values, separately: each of the $\mu_k$ depends on $\theta$ alone

* Our simple Poisson model reflects an assumption about our detector, which is that the observed counts in the $k^{th}$ pixel are independent of the observed counts in all the other pixels. $N_k$ only depends on its pixel's expected counts $\mu_k$

* In this case, the sampling distribution for all the observed pixel values $\boldsymbol{N}$ factorizes and simplifies to

$\;\;\;\;\;\;\;\;\;\;P(\boldsymbol{N}|\boldsymbol{\mu},H) = \prod_k P(N_k|\mu_k,H)$

## All pixels, fixed inputs

<img src="../graphics/pgms_all_pixels_input_fixed.png">

$\;\;\;\;\;\;\;P(\boldsymbol{N},\boldsymbol{\mu},\theta|H) = \left[ \prod_k P(N_k|\mu_k,H)  P(\mu_k|\theta,H) \right] P(\theta|H)$

## PGM pieces

* Each _node_ in the graph represents a PDF, for the variable labeled inside it

* Each _edge_ (arrow) in the graph represents a conditional dependence

* Deterministic relationships are indicated by "fixed" variables represented by solid points

* Plates contain conditionally independent variables

> Think of plates as illustrating a stack of layers, seen from above, that are only connected by the arrows coming from variables outside the plate.

## Take-home messages

* Generating data is the key function of a statistical model

* Noise-free mock datasets are central to inference; noisy mock datasets are vital for testing and model checking

* Generating a mock dataset means drawing a sample from the joint probability distribution for all variables in the model

* Probabilstic Graphical Models (PGMs) illustrate a particular factorization of this joint PDF, and can be viewed as "flow charts" for data simulation process

* The sampling distribution captures the statistical uncertainties in the data; drawing from it "adds noise" to make the final mock dataset




**Take-home messages**

* Both simulation of mock data and model inference from data require a model for how the Universe (or our computer) generates data.
* PGMs are a helpful way of visualizing the conditional dependences of a model (how the probability expressions factorize).

Note: the `daft` Python package is useful for making pretty PGMs.

## Coded Examples

In the `examples` folder, there are some notebooks that show:

* [A "catalog" dataset from the Sloan Digital Sky Survey](../examples/SDSScatalog/FirstLook.ipynb)

* [An attempt to generate a mock dataset](../examples/SDSScatalog/GalaxySizes.ipynb) that (literally) looks like the real one

* [How the `daft` python package can be used to draw PGMs](../examples/SDSScatalog/FirstPGM.ipynb)


You might (one day) find some of this code useful.

## Practice

### "Standard" linear regression

You're given a list of $x_k,y_k,\sigma_k$ triplets, where $\sigma_k$ is some estimate of the "error" on $y_k$. You'd like to fit a linear model, $y(x)=a+bx$. In the absence of any better information, assume that $\vec{x}$ and $\vec{\sigma}$ are (somehow) known precisely, and that the "error" on $y_k$ is Gaussian (mean of $a+bx_k$ and standard deviation $\sigma_k$).

1. Draw the PGM and write down the corresponsing probability expressions for this problem.
2. Simulate a few data sets, given some values (your choice) for the input parameters. The code below is a (crummy) starting point.
3. What (unspecified) assumptions, if any, did you have to make? Which assumptions do you think are unlikely to hold in practice? Choose one (or more) of these assumptions and work out how to generalize the PGM/generative model to avoid making it.

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
plt.rcParams['xtick.labelsize'] = 'x-large'
plt.rcParams['ytick.labelsize'] = 'x-large'
%matplotlib inline

In [ ]:
# choose some linear model parameters, somehow
a = 
b = 

# choose some x and sigma values... somehow
n = 10 # Number of data points. Feel free to change.
x = np.array([
sigma = np.array([

# work out the values for any intermediate nodes
    
# generate the "observed" y values
y = st.norm.rvs(

In [ ]:
# plot x, y and sigma in the usual way
plt.rcParams['figure.figsize'] = (12.0, 5.0)
plt.errorbar(x, y, yerr=sigma, fmt='none');
plt.xlabel('x', fontsize=14);
plt.ylabel('y', fontsize=14);

### Exoplanet transit photometry

You've taken images of a particular field on several nights, in order to record the transit of an exoplanet in front of a star (resulting in a temporary decrease in its brightness). Some kind of model, parametrized by $\theta$, describes the time series of the resulting flux. Before we get to measure a number of counts, however, each image is affected by night-specific variables such as weather. To account for these, you've also measured 10 other stars in the same field. The assumption is that the average intrinsic flux of these stars should be constant in time, so that they can be used to correct for nightly photometric variations, putting the multiple measurements of the target star on the same scale.

Draw the PGM and write down the corresponsing probability expressions for this problem.

Thanks to Anja von der Linden for inspiring the above problem.

### Galaxy cluster center offsets

You've measured the centers of a sample of galaxy clusters in two ways: by choosing a brightest cluster galaxy (BCG) and by finding the centroid of each cluster's X-ray emission. The difference between the two should say something about the fidelity of the BCG selection method, among other things. The BCG positions are determined essentially perfectly, but the X-ray centroids come with a Gaussian statistical uncertainty of typically $\sim30$ kpc (standard deviation) in both the $x$ and $y$ directions.

The underlying model is assumed to be that the BCG and true X-ray centroid coincide perfectly in a fraction $f$ of clusters. In the remaining clusters, the true X-ray centroid and BCG are displaced according to a 2D Gaussian whose width in either direction is $\sigma=100$ kpc.

1. Draw the PGM and write down the corresponsing probability expressions for this problem.
2. Simulate some data sets and visualize them, e.g. as a histogram of the offset distances.

Thanks to the bloody Universe for inspiring the above problem.